In [1]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
import dateutil.parser as dp
from datetime import datetime
from datetime import timedelta  
import time
from datetime import timezone
session = Session()
import os

import numpy as np

#converts pyton datatime to unix milisecond timestamp utc timezone
def to_utcunix(x):
    return int(x.replace(tzinfo=timezone.utc).timestamp()) * 1000

#Generator for binance price data
def get_data(start, batch_size, end = datetime.now(), symbol = "BTCUSDT", interval = "1m"):
    batch_start = start
    
    while True:
        batch_end = batch_start + timedelta(minutes = batch_size)
        
        if(batch_end > end):
            batch_end = end - timedelta(minutes = 1)
        
            if(batch_start >= end- timedelta(minutes = 1)):
                break
        
        base = "https://api.binance.com/api/v3"

        print(batch_start, batch_end)
        
        url = base + "/klines?symbol={}&interval={}&startTime={}&endTime={}&limit={}".format(symbol, interval,
                                                                                     to_utcunix(batch_start)
                                                                                    , to_utcunix(batch_end), batch_size)
        
        
        batch_start = batch_end
        #print(url)
        #sleep to stay under binance maximum call rate
        time.sleep(0.15)
        
        while True:
            try:
                data = session.get(url).json()
                #print(data)
                yield pd.DataFrame.from_dict(data)[[0,1,2,3,4,5]]
            except:
                print("Retrieval error")
                yield pd.DataFrame(np.zeros([batch_size, 6]))
                
                #time.sleep(10)
                #continue
            break

#save binance price data as csv
def gen_csv(start, filename, end = datetime.now(), symbol = "BTCUSDT", interval = "1m"):
    batch_size = 1000
    d = get_data(start, batch_size, end = end, symbol = symbol, interval = interval)
        
    try:
        os.mkdir("data_" + interval)
    except:
        pass
    
    fn = "data_" + interval + "/" + filename + ".csv"
    pd.DataFrame([["Open Time", "Open", "High",  "Low",  "Close",  "Volume"]]).to_csv(fn, mode = 'a', header = None)
    for i in d:
        i.to_csv(fn, mode = 'a', header = None)#, header = ["Open Time", "Open", "High",  "Low",  "Close",  "Volume"])

#simplified gen_csv function
def gen_csv_year(year, symbol, interval = "1m"):
    gen_csv(datetime(year,1,1) , "{}_{}".format(year, symbol) ,datetime(year + 1,1,1), symbol, interval )
    

#for i in range(2016, 2021):
#    gen_csv_year(i, "ETHUSDT")    


In [ ]:

#for s in ['ETHUSDT', 'BTCUSDT']:
#    gen_csv_year(2018, s, interval = "5m") 

for y in range(2018, 2021):
    for s in ['XRPUSDT', 'BCHUSDT', "LTCUSDT"]:
        gen_csv_year(y, s, interval = "5m") 

2018-01-01 00:00:00 2018-01-01 16:40:00
Retrieval error
2018-01-01 16:40:00 2018-01-02 09:20:00
Retrieval error
2018-01-02 09:20:00 2018-01-03 02:00:00
Retrieval error
2018-01-03 02:00:00 2018-01-03 18:40:00
Retrieval error
2018-01-03 18:40:00 2018-01-04 11:20:00
Retrieval error
2018-01-04 11:20:00 2018-01-05 04:00:00
Retrieval error
2018-01-05 04:00:00 2018-01-05 20:40:00
Retrieval error
2018-01-05 20:40:00 2018-01-06 13:20:00
Retrieval error
2018-01-06 13:20:00 2018-01-07 06:00:00
Retrieval error
2018-01-07 06:00:00 2018-01-07 22:40:00
Retrieval error
2018-01-07 22:40:00 2018-01-08 15:20:00
Retrieval error
2018-01-08 15:20:00 2018-01-09 08:00:00
Retrieval error
2018-01-09 08:00:00 2018-01-10 00:40:00
Retrieval error
2018-01-10 00:40:00 2018-01-10 17:20:00
Retrieval error
2018-01-10 17:20:00 2018-01-11 10:00:00
Retrieval error
2018-01-11 10:00:00 2018-01-12 02:40:00
Retrieval error
2018-01-12 02:40:00 2018-01-12 19:20:00
Retrieval error
2018-01-12 19:20:00 2018-01-13 12:00:00
Retrieva

Retrieval error
2018-04-13 02:00:00 2018-04-13 18:40:00
Retrieval error
2018-04-13 18:40:00 2018-04-14 11:20:00
Retrieval error
2018-04-14 11:20:00 2018-04-15 04:00:00
Retrieval error
2018-04-15 04:00:00 2018-04-15 20:40:00
Retrieval error
2018-04-15 20:40:00 2018-04-16 13:20:00
Retrieval error
2018-04-16 13:20:00 2018-04-17 06:00:00
Retrieval error
2018-04-17 06:00:00 2018-04-17 22:40:00
Retrieval error
2018-04-17 22:40:00 2018-04-18 15:20:00
Retrieval error
2018-04-18 15:20:00 2018-04-19 08:00:00
Retrieval error
2018-04-19 08:00:00 2018-04-20 00:40:00
Retrieval error
2018-04-20 00:40:00 2018-04-20 17:20:00
Retrieval error
2018-04-20 17:20:00 2018-04-21 10:00:00
Retrieval error
2018-04-21 10:00:00 2018-04-22 02:40:00
Retrieval error
2018-04-22 02:40:00 2018-04-22 19:20:00
Retrieval error
2018-04-22 19:20:00 2018-04-23 12:00:00
Retrieval error
2018-04-23 12:00:00 2018-04-24 04:40:00
Retrieval error
2018-04-24 04:40:00 2018-04-24 21:20:00
Retrieval error
2018-04-24 21:20:00 2018-04-25 1

2018-08-25 02:40:00 2018-08-25 19:20:00
2018-08-25 19:20:00 2018-08-26 12:00:00
2018-08-26 12:00:00 2018-08-27 04:40:00
2018-08-27 04:40:00 2018-08-27 21:20:00
2018-08-27 21:20:00 2018-08-28 14:00:00
2018-08-28 14:00:00 2018-08-29 06:40:00
2018-08-29 06:40:00 2018-08-29 23:20:00
2018-08-29 23:20:00 2018-08-30 16:00:00
2018-08-30 16:00:00 2018-08-31 08:40:00
2018-08-31 08:40:00 2018-09-01 01:20:00
2018-09-01 01:20:00 2018-09-01 18:00:00
2018-09-01 18:00:00 2018-09-02 10:40:00
2018-09-02 10:40:00 2018-09-03 03:20:00
2018-09-03 03:20:00 2018-09-03 20:00:00
2018-09-03 20:00:00 2018-09-04 12:40:00
2018-09-04 12:40:00 2018-09-05 05:20:00
2018-09-05 05:20:00 2018-09-05 22:00:00
2018-09-05 22:00:00 2018-09-06 14:40:00
2018-09-06 14:40:00 2018-09-07 07:20:00
2018-09-07 07:20:00 2018-09-08 00:00:00
2018-09-08 00:00:00 2018-09-08 16:40:00
2018-09-08 16:40:00 2018-09-09 09:20:00
2018-09-09 09:20:00 2018-09-10 02:00:00
2018-09-10 02:00:00 2018-09-10 18:40:00
2018-09-10 18:40:00 2018-09-11 11:20:00


Retrieval error
2018-01-10 17:20:00 2018-01-11 10:00:00
Retrieval error
2018-01-11 10:00:00 2018-01-12 02:40:00
Retrieval error
2018-01-12 02:40:00 2018-01-12 19:20:00
Retrieval error
2018-01-12 19:20:00 2018-01-13 12:00:00
Retrieval error
2018-01-13 12:00:00 2018-01-14 04:40:00
Retrieval error
2018-01-14 04:40:00 2018-01-14 21:20:00
Retrieval error
2018-01-14 21:20:00 2018-01-15 14:00:00
Retrieval error
2018-01-15 14:00:00 2018-01-16 06:40:00
Retrieval error
2018-01-16 06:40:00 2018-01-16 23:20:00
Retrieval error
2018-01-16 23:20:00 2018-01-17 16:00:00
Retrieval error
2018-01-17 16:00:00 2018-01-18 08:40:00
Retrieval error
2018-01-18 08:40:00 2018-01-19 01:20:00
Retrieval error
2018-01-19 01:20:00 2018-01-19 18:00:00
Retrieval error
2018-01-19 18:00:00 2018-01-20 10:40:00
Retrieval error
2018-01-20 10:40:00 2018-01-21 03:20:00
Retrieval error
2018-01-21 03:20:00 2018-01-21 20:00:00
Retrieval error
2018-01-21 20:00:00 2018-01-22 12:40:00
Retrieval error
2018-01-22 12:40:00 2018-01-23 0

Retrieval error
2018-04-22 19:20:00 2018-04-23 12:00:00
Retrieval error
2018-04-23 12:00:00 2018-04-24 04:40:00
Retrieval error
2018-04-24 04:40:00 2018-04-24 21:20:00
Retrieval error
2018-04-24 21:20:00 2018-04-25 14:00:00
Retrieval error
2018-04-25 14:00:00 2018-04-26 06:40:00
Retrieval error
2018-04-26 06:40:00 2018-04-26 23:20:00
Retrieval error
2018-04-26 23:20:00 2018-04-27 16:00:00
Retrieval error
2018-04-27 16:00:00 2018-04-28 08:40:00
Retrieval error
2018-04-28 08:40:00 2018-04-29 01:20:00
Retrieval error
2018-04-29 01:20:00 2018-04-29 18:00:00
Retrieval error
2018-04-29 18:00:00 2018-04-30 10:40:00
Retrieval error
2018-04-30 10:40:00 2018-05-01 03:20:00
Retrieval error
2018-05-01 03:20:00 2018-05-01 20:00:00
Retrieval error
2018-05-01 20:00:00 2018-05-02 12:40:00
Retrieval error
2018-05-02 12:40:00 2018-05-03 05:20:00
Retrieval error
2018-05-03 05:20:00 2018-05-03 22:00:00
Retrieval error
2018-05-03 22:00:00 2018-05-04 14:40:00
Retrieval error
2018-05-04 14:40:00 2018-05-05 0

Retrieval error
2018-08-02 21:20:00 2018-08-03 14:00:00
Retrieval error
2018-08-03 14:00:00 2018-08-04 06:40:00
Retrieval error
2018-08-04 06:40:00 2018-08-04 23:20:00
Retrieval error
2018-08-04 23:20:00 2018-08-05 16:00:00
Retrieval error
2018-08-05 16:00:00 2018-08-06 08:40:00
Retrieval error
2018-08-06 08:40:00 2018-08-07 01:20:00
Retrieval error
2018-08-07 01:20:00 2018-08-07 18:00:00
Retrieval error
2018-08-07 18:00:00 2018-08-08 10:40:00
Retrieval error
2018-08-08 10:40:00 2018-08-09 03:20:00
Retrieval error
2018-08-09 03:20:00 2018-08-09 20:00:00
Retrieval error
2018-08-09 20:00:00 2018-08-10 12:40:00
Retrieval error
2018-08-10 12:40:00 2018-08-11 05:20:00
Retrieval error
2018-08-11 05:20:00 2018-08-11 22:00:00
Retrieval error
2018-08-11 22:00:00 2018-08-12 14:40:00
Retrieval error
2018-08-12 14:40:00 2018-08-13 07:20:00
Retrieval error
2018-08-13 07:20:00 2018-08-14 00:00:00
Retrieval error
2018-08-14 00:00:00 2018-08-14 16:40:00
Retrieval error
2018-08-14 16:40:00 2018-08-15 0

Retrieval error
2018-11-12 23:20:00 2018-11-13 16:00:00
Retrieval error
2018-11-13 16:00:00 2018-11-14 08:40:00
Retrieval error
2018-11-14 08:40:00 2018-11-15 01:20:00
Retrieval error
2018-11-15 01:20:00 2018-11-15 18:00:00
Retrieval error
2018-11-15 18:00:00 2018-11-16 10:40:00
Retrieval error
2018-11-16 10:40:00 2018-11-17 03:20:00
Retrieval error
2018-11-17 03:20:00 2018-11-17 20:00:00
Retrieval error
2018-11-17 20:00:00 2018-11-18 12:40:00
Retrieval error
2018-11-18 12:40:00 2018-11-19 05:20:00
Retrieval error
2018-11-19 05:20:00 2018-11-19 22:00:00
Retrieval error
2018-11-19 22:00:00 2018-11-20 14:40:00
Retrieval error
2018-11-20 14:40:00 2018-11-21 07:20:00
Retrieval error
2018-11-21 07:20:00 2018-11-22 00:00:00
Retrieval error
2018-11-22 00:00:00 2018-11-22 16:40:00
Retrieval error
2018-11-22 16:40:00 2018-11-23 09:20:00
Retrieval error
2018-11-23 09:20:00 2018-11-24 02:00:00
Retrieval error
2018-11-24 02:00:00 2018-11-24 18:40:00
Retrieval error
2018-11-24 18:40:00 2018-11-25 1

2018-03-14 22:00:00 2018-03-15 14:40:00
2018-03-15 14:40:00 2018-03-16 07:20:00
2018-03-16 07:20:00 2018-03-17 00:00:00
2018-03-17 00:00:00 2018-03-17 16:40:00
2018-03-17 16:40:00 2018-03-18 09:20:00
2018-03-18 09:20:00 2018-03-19 02:00:00
2018-03-19 02:00:00 2018-03-19 18:40:00
2018-03-19 18:40:00 2018-03-20 11:20:00
2018-03-20 11:20:00 2018-03-21 04:00:00
2018-03-21 04:00:00 2018-03-21 20:40:00
2018-03-21 20:40:00 2018-03-22 13:20:00
2018-03-22 13:20:00 2018-03-23 06:00:00
2018-03-23 06:00:00 2018-03-23 22:40:00
2018-03-23 22:40:00 2018-03-24 15:20:00
2018-03-24 15:20:00 2018-03-25 08:00:00
2018-03-25 08:00:00 2018-03-26 00:40:00
2018-03-26 00:40:00 2018-03-26 17:20:00
2018-03-26 17:20:00 2018-03-27 10:00:00
2018-03-27 10:00:00 2018-03-28 02:40:00
2018-03-28 02:40:00 2018-03-28 19:20:00
2018-03-28 19:20:00 2018-03-29 12:00:00
2018-03-29 12:00:00 2018-03-30 04:40:00
2018-03-30 04:40:00 2018-03-30 21:20:00
2018-03-30 21:20:00 2018-03-31 14:00:00
2018-03-31 14:00:00 2018-04-01 06:40:00
